We will use fiona to read the shapefile and pyproj to transform it from one coordinate system to another. 

In [9]:
import os
import glob
import fiona
from fiona.crs import from_epsg
from pprint import pprint
from pyproj import Proj, transform

In [11]:
workdir = os.getcwd()
indir = os.path.join(workdir, 'Outdata', 'Buffer_a_Line')
inFileName = 'Buffered_Adams_Is.shp'
inFilePath = os.path.join(indir, inFileName)
outFileName = 'Projected_Adams_Is.shp'
outFilePath = os.path.join(indir, outFileName)

In [28]:
with fiona.open(inFilePath, 'r') as src:
    incrs = src.crs
    inproj = Proj(incrs, preserve_units=True)
    outproj = Proj(fiona.crs.from_epsg(26915))
    with fiona.open(outFilePath,'w', src.driver, src.schema.copy(), crs=outproj.srs) as dest:
        for feature in src:
            coordinates = feature['geometry']['coordinates'][0]
            new_coordinates = []
            for coords in coordinates:
                x, y = coords
                new_x, new_y = transform(inproj, outproj, x, y)
                new_coordinates.append((new_x, new_y))
            feature['geometry']['coordinates'][0] = new_coordinates
            dest.write(feature)

In [34]:
with fiona.open(inFilePath, 'r') as src1:
    pprint(src1.crs_wkt)

'PROJCS["NAD_1983_StatePlane_Illinois_West_FIPS_1202_Feet",GEOGCS["GCS_North_American_1983",DATUM["North_American_Datum_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.017453292519943295]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",2296583.333333333],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-90.16666666666667],PARAMETER["Scale_Factor",0.9999411764705882],PARAMETER["Latitude_Of_Origin",36.66666666666666],UNIT["Foot_US",0.30480060960121924]]'


In [33]:
with fiona.open(outFilePath, 'r') as src2:
    pprint(src2.crs_wkt)

'PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["North_American_Datum_1983",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-93],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["Meter",1]]'
